In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000029265BCC4A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029265BCD6A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi My name is Aakash and i am an AI engineer")])

AIMessage(content="Hi Aakash, it's nice to meet you!\n\nThat's awesome that you're an AI engineer. What kind of projects are you working on?  \n\nI'm always interested to learn more about what people are doing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 22, 'total_tokens': 81, 'completion_time': 0.107272727, 'prompt_time': 0.002127517, 'queue_time': 0.242674942, 'total_time': 0.109400244}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--35ab7ca5-e85b-4373-a906-a868c6daff40-0', usage_metadata={'input_tokens': 22, 'output_tokens': 59, 'total_tokens': 81})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi My name is Aakash and i am an AI engineer"),
        AIMessage(content="Hi Aakash, it's nice to meet you!\n\nThat's awesome that you're an AI engineer. What kind of projects are you working on?"),
        HumanMessage(content="Hey, whats my name and what do i do?")
    ]
)

AIMessage(content="According to our conversation, your name is Aakash and you are an AI engineer. 😊 \n\nIs there anything else you'd like to talk about?  Maybe you could tell me more about your work as an AI engineer!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 78, 'total_tokens': 129, 'completion_time': 0.092727273, 'prompt_time': 0.003949164, 'queue_time': 0.252670496, 'total_time': 0.096676437}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cfd8fbdd-12b5-441a-a226-047fa592b204-0', usage_metadata={'input_tokens': 78, 'output_tokens': 51, 'total_tokens': 129})

## MESSAGE HISTORY

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [8]:
config = {"configurable":{"session_id":"chat1"}}

In [10]:
response = with_message_history.invoke([HumanMessage(content="Hi My name is Aakash and i am an AI engineer")],
                            config=config)

In [11]:
response.content

"Hi Aakash, it's great to meet you!  \n\nBeing an AI engineer is really exciting. What kind of projects are you working on right now? Are you focused on a particular area like natural language processing, computer vision, or something else?  \n\nI'm always curious to hear about the cool things people are building with AI. 😊\n"

In [12]:
# Change config --> change the session id
config1 = {"configurable":{"session_id":"chat2"}}
res = with_message_history.invoke([HumanMessage(content="whats my name")],config=config1)
res.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn! 😊\n"

### PROMPT TEMPLATES


In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a h  elpful assistant.Answer all questios to best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Aakash")]})

AIMessage(content="Hi Aakash! It's nice to meet you. 😊\n\nI'm happy to help with any questions you have. Just ask away!  What can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_time': 0.081818182, 'prompt_time': 0.002561944, 'queue_time': 0.24608814399999998, 'total_time': 0.084380126}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--65fb6941-009d-40f7-888d-e2e48580a287-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, 'total_tokens': 79})

In [15]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke([HumanMessage(content="Hi My name is Aakash")],config = config)

response

AIMessage(content="Hi Aakash, it's nice to meet you!  \n\nWhat can I help you with today?  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 34, 'total_tokens': 64, 'completion_time': 0.054545455, 'prompt_time': 0.002381795, 'queue_time': 0.25015405399999996, 'total_time': 0.05692725}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--489fa304-587d-4a51-bf0a-7e8d3392b6d2-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64})

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a h  elpful assistant.Answer all questios to best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [18]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Aakash")],"language":"Hindi"})
response.content

'नमस्ते आकाश!  \n\nमुझे आपकी मदद करने में खुशी होगी। आप किस बारे में जानना चाहते हैं?  \n\n'

In [19]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [ ]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke({"messages":[HumanMessage(content="Hi My name is Aakash")],
                                       "language":"hindi"},
                                       config = config)

response

SyntaxError: invalid syntax (2451969221.py, line 2)